#Features extracting from audio

Create spectogram for each audio file and get feature matrix from it

In [2]:
import os
import glob
os.chdir('/Users/vidmantasbendikas/Documents/github/soundrecognition')
audio_folder = 'audio_preparation/Audio_splitted'
spectrograms_folder = 'audio_preparation/spectrograms'

if not os.path.exists(spectrograms_folder):
    os.makedirs(spectrograms_folder)


list_subdirs = os.listdir(audio_folder)
s=0
LOG_EVERY_N = 100
for i in list_subdirs:
    s += 1
    if (s % LOG_EVERY_N) == 0:
        print "Preprocessed "+str(s)+" people records"  
    directory = audio_folder+'/'+i+'/'
    new_directory = spectrograms_folder+'/'+i+'/'
    if not os.path.exists(new_directory):
        os.makedirs(new_directory)
    #
    list_wav = glob.glob(directory+"*.wav")
    for k in range(len(list_wav)):
        spectrogram_dir = spectrograms_folder+'/'+i+'/'+ i+'_'+list_wav[k].split('/')[-1].split('.')[0]+'.png'
        #command = "sox "+list_wav[k]+" -n spectrogram -l -r -o "+spectrogram_dir
        command = "sox "+list_wav[k]+" -n spectrogram -l -m -r -w Kaiser -o "+spectrogram_dir
        os.system(command)

Preprocessed 100 audio files


Preprocessed 200 audio files


Preprocessed 300 audio files


Preprocessed 400 audio files


Preprocessed 500 audio files


Transform spectrogram image into matrix and save it to tuple

In [2]:
import os.path
from PIL import Image
import numpy as np
import pickle

def img_to_matrix(filename, verbose=False, STANDARD_SIZE= (120,120)):
    """
    takes a filename and turns it into a numpy array of RGB pixels
    """
    img = Image.open(filename)
    if verbose==True:
        print "changing size from %s to %s" % (str(img.size), str(STANDARD_SIZE))
    img = img.resize(STANDARD_SIZE)
    img = list(img.getdata())
    img = np.array(img)
    return img

audios_features_folder = 'audios_features'
spectograms_subfolders = os.listdir(spectrograms_folder)
audios_features = {}
s = 0
for subfolder in spectograms_subfolders:
    s += 1
    if (s % LOG_EVERY_N) == 0:
        print "Transformed "+str(s)+" spectrograms into array"  
    path_to_spectograms_files = spectrograms_folder+'/'+subfolder+'/'
    spectrograms_files_list = glob.glob(path_to_spectograms_files+"*.png")
    
    audios_features[subfolder] = {}
    audios_features[subfolder]['device'] = {}
    audios_features[subfolder]['device']['studio'] = {}
    audios_features[subfolder]['device']['iphone'] = {}
    audios_features[subfolder]['device']['samsung'] = {}
    for spectrogram_file in spectrograms_files_list:
        if spectrogram_file.split('_')[-1].split('a')[0]=='s':
            audio_features = img_to_matrix(spectrogram_file)
            audios_features[subfolder]['device']['samsung'][spectrogram_file.split('/')[-1]] = {}
            audios_features[subfolder]['device']['samsung'][spectrogram_file.split('/')[-1]]['features'] = {}
            audios_features[subfolder]['device']['samsung'][spectrogram_file.split('/')[-1]]['features']['spectrogram'] = audio_features
       
        elif spectrogram_file.split('_')[-1].split('a')[0]=='i':
            audio_features = img_to_matrix(spectrogram_file)
            audios_features[subfolder]['device']['iphone'][spectrogram_file.split('/')[-1]] = {}
            audios_features[subfolder]['device']['iphone'][spectrogram_file.split('/')[-1]]['features'] = {}
            audios_features[subfolder]['device']['iphone'][spectrogram_file.split('/')[-1]]['features']['spectrogram'] = audio_features 
        else:
            audio_features = img_to_matrix(spectrogram_file)
            audios_features[subfolder]['device']['studio'][spectrogram_file.split('/')[-1]] = {}
            audios_features[subfolder]['device']['studio'][spectrogram_file.split('/')[-1]]['features'] = {}
            audios_features[subfolder]['device']['studio'][spectrogram_file.split('/')[-1]]['features']['spectrogram'] = audio_features

if not os.path.exists(audios_features_folder):
    os.makedirs(audios_features_folder)

pickle.dump(audios_features, open(audios_features_folder+"/"+"audios_features.p", "wb" ))

OSError: [Errno 2] No such file or directory: 'audio_preparation/spectrograms'